In [1]:
# Import necessary libraries
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input
from keras.layers.noise import AlphaDropout
from keras.layers import Layer
from tensorflow.python.keras import backend as K
from sklearn.model_selection import train_test_split
import pickle

# Import custom modules
from network import *
from data import *

/Users/dweepa/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dbfile = open('../../Data/full', 'rb')    
data = pickle.load(dbfile)
dbfile = open('../../Data/location_pert', 'rb')    
location_pert = pickle.load(dbfile)
dbfile = open('../../Data/pert2profiles', 'rb')    
pert2profiles = pickle.load(dbfile)
dbfile = open('../../Data/test_perts', 'rb')    
test_pert = pickle.load(dbfile)
dbfile = open('../../Data/train_perts', 'rb')    
train_pert = pickle.load(dbfile)
dbfile = open('../../Data/X_test_triplet', 'rb')
X_test = pickle.load(dbfile)
dbfile = open('../../Data/X_train_triplet', 'rb')
X_train = pickle.load(dbfile)
dbfile.close()

In [3]:
X_train = generate_data(data,train_pert[0:100],100) 

batch_size:  10000
71/10000

KeyboardInterrupt: 

# Network

In [3]:
# Create the base network
network = dict(n_dense=5, dense_units=16, activation='selu', dropout=AlphaDropout, dropout_rate=0.1,
               kernel_initializer='lecun_normal', optimizer='sgd', num_classes=2)

shared_model = create_base_network(**network)

In [4]:
# Create the siamese network
model = create_siamese_network(shared_model)
print("triplet network model summary")
model.summary()

y_pred.shape =  Tensor("merged_layer/concat:0", shape=(?, 48), dtype=float32)
Tensor("loss/merged_layer_loss/Abs:0", shape=(?,), dtype=float32)
Tensor("loss/merged_layer_loss/Abs_1:0", shape=(?,), dtype=float32)
triplet network model summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 978)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 978)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 978)          0                                            
_________________________________________________________________

In [5]:
index = np.random.choice(X_train.shape[1],84, replace=False)  
x_test_anch = X_train[0][index]
x_test_pos = X_train[1][index]
x_test_neg = X_train[2][index]

def Diff(li1, li2):
    li_dif = [i for i in li1 if i not in li1 or i not in li2]
    return li_dif
index2 = Diff([i for i in range(X_train.shape[1])], index)

x_train_anch = X_train[0][index2]
x_train_pos = X_train[1][index2]
x_train_neg = X_train[2][index2]

In [6]:
x_train_anch.shape

(336, 978)

In [7]:
#y_dum = np.zeros((len(X_train[0]),1))
#model.fit([X_train[0],X_train[1],X_train[2]],y_dum,epochs=100)
y_dum = np.zeros((len(x_train_anch),1))
model.fit([x_train_anch,x_train_anch,x_train_anch],y_dum,epochs=100)

Epoch 1/100
336/336 [==============================] - 1s 4ms/step - loss: 5.8792
Epoch 2/100
336/336 [==============================] - 0s 172us/step - loss: 4.7468
Epoch 3/100
336/336 [==============================] - 0s 164us/step - loss: 4.8084
Epoch 4/100
336/336 [==============================] - 0s 174us/step - loss: 4.4842
Epoch 5/100
336/336 [==============================] - 0s 164us/step - loss: 3.8604
Epoch 6/100
336/336 [==============================] - 0s 171us/step - loss: 4.1187
Epoch 7/100
336/336 [==============================] - 0s 171us/step - loss: 3.9424
Epoch 8/100
336/336 [==============================] - 0s 172us/step - loss: 3.1798
Epoch 9/100
336/336 [==============================] - 0s 165us/step - loss: 3.3279
Epoch 10/100
336/336 [==============================] - 0s 173us/step - loss: 2.9847
Epoch 11/100
336/336 [==============================] - 0s 167us/step - loss: 2.8031
Epoch 12/100
336/336 [==============================] - 0s 170us/step - loss

In [8]:
pred = model.predict([x_test_anch,x_test_pos,x_test_neg])
pred

array([[-1.583947 , -1.6194172, -1.5862105, ..., -1.536695 , -1.5743308,
        -1.6507661],
       [-1.5757152, -1.6363274, -1.5680358, ..., -1.5520461, -1.585606 ,
        -1.6303561],
       [-1.5731959, -1.6144158, -1.569976 , ..., -1.4694622, -1.5734942,
        -1.7003508],
       ...,
       [-1.5529535, -1.6310512, -1.5700498, ..., -1.5251944, -1.5872489,
        -1.6762955],
       [-1.5860906, -1.6189401, -1.584371 , ...,  0.3392265,  1.3834811,
         2.0342476],
       [-1.5765158, -1.639289 , -1.5760888, ..., -1.5867943, -1.530628 ,
        -1.5978836]], dtype=float32)

In [9]:
from sklearn.preprocessing import normalize
anch = normalize(pred[0][0:16].reshape(1,-1)).flatten()
pos = normalize(pred[0][16:32].reshape(1,-1)).flatten()
neg = normalize(pred[0][32:].reshape(1,-1)).flatten()

In [10]:
values = []
for i in pred:
    anch = normalize(i[0:16].reshape(1,-1)).flatten()
    pos = normalize(i[16:32].reshape(1,-1)).flatten()
    neg = normalize(i[32:].reshape(1,-1)).flatten()
    pos_neg_val = [(np.dot(anch,pos)), (np.dot(anch,neg))]
    print(pos_neg_val)
    values.append(pos_neg_val[0]>=0.7)
    values.append(pos_neg_val[1]<0.7)
    # print(positives, negatives)
    #     print(pos_val, neg_val, pos_val>neg_val)
print((np.sum(np.asarray(values)==True))/len(values))

[0.999987, 0.9998356]
[0.9998348, 0.9998255]
[0.9998568, 0.99980605]
[0.9998107, 0.9999363]
[0.999984, 0.9998554]
[0.9999171, 0.9999514]
[0.9999774, 0.99967086]
[0.999859, 0.99960554]
[0.9998547, 0.9998702]
[0.99994856, 0.9999565]
[0.9999229, 0.9999172]
[0.99997616, 0.9998562]
[0.999959, 0.99997956]
[0.9999441, 0.99986583]
[0.9998956, 0.99998957]
[0.99988127, 0.9999088]
[0.9998465, 0.9999114]
[0.9999581, 0.99991316]
[0.99994946, 0.99989176]
[0.9999647, 0.99978745]
[0.99996585, 0.999953]
[0.9999327, -0.8265517]
[0.999984, 0.9999844]
[0.99994826, 0.99992824]
[0.99998224, 0.9998656]
[0.99999666, 0.9997834]
[0.99998295, 0.9998973]
[0.99988127, 0.9997379]
[0.99995166, 0.99979293]
[0.99984515, 0.99990547]
[0.9997624, 0.99963707]
[0.9997292, 0.999798]
[0.9998716, 0.99993145]
[0.9997915, 0.999857]
[0.9999031, 0.99998236]
[0.9999031, 0.9998052]
[0.99997365, 0.99963856]
[0.99996346, 0.9998785]
[0.9996818, 0.9998441]
[0.99976176, 0.99969846]
[0.9999489, 0.99995446]
[0.99994767, 0.9998245]
[0.9999

In [11]:
pos_neg_val

[0.99997103, 0.99976456]